# Fine-tunning de BETO para predicción de Emojis

In [1]:
from config import *

Primero debemos importar el dataset. En nuestro caso este está guardado como un pandas pickleado (fuente: [este tutorial](https://huggingface.co/course/chapter5/2)). La biblioteca [datasets](https://huggingface.co/docs/datasets/index) nos permite cargar un dataset con este formato. Los path pueden ser modificados en el archivo config.py.

In [2]:
from datasets import load_dataset

# data_files = {"train": file_names['df_es_train'], "test": file_names['df_es_test'], "trial": file_names['df_es_trial']}
data_files = {"train": file_names['df_es_trial'], "test": file_names['df_es_test']}  # a modo de testeo no entrenaremos en el train set entero
dataset_emoji = load_dataset("pandas", data_files=data_files)
dataset_emoji

Using custom data configuration default-cb077f95137f42ce
Reusing dataset pandas (/home/camilo/.cache/huggingface/datasets/pandas/default-cb077f95137f42ce/0.0.0/6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade)
100%|██████████| 2/2 [00:00<00:00, 973.83it/s]


DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['id', 'text', 'label'],
        num_rows: 10000
    })
})

In [3]:
dataset_emoji = dataset_emoji.remove_columns("id")

In [4]:
dataset_emoji["train"][0]

{'text': 'Plaza de Oriente , Madrid .......#madrid #city #plazadeoriente #puertadesol #tour…',
 'label': '9'}

In [5]:
dataset_emoji["train"].features

{'text': Value(dtype='string', id=None),
 'label': Value(dtype='string', id=None)}

In [6]:
from datasets import ClassLabel

dataset_emoji = dataset_emoji.cast_column("label",ClassLabel(num_classes=19))

Loading cached processed dataset at /home/camilo/.cache/huggingface/datasets/pandas/default-cb077f95137f42ce/0.0.0/6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade/cache-d9b1254ac31c380b.arrow
Loading cached processed dataset at /home/camilo/.cache/huggingface/datasets/pandas/default-cb077f95137f42ce/0.0.0/6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade/cache-e3006495028cdd71.arrow


In [7]:
dataset_emoji["train"].features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(num_classes=19, names=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18'], id=None)}

Queremos hacer finetuning a BETO para la tarea de predecir emojis. En este caso tendremos un conjunto fijo de emojis, por ende se trata de una tarea de clasificación. Seguiremos partes de [este tutorial](https://huggingface.co/docs/transformers/tasks/sequence_classification).

### Importando el tokenizador

In [8]:
from transformers import AutoTokenizer

model_id = "dccuchile/bert-base-spanish-wwm-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_id)

Necesitamos crear una función que tokenice los elementos del dataset

In [9]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)

Notemos que no necesitamos separar los hashtags, pues el tokenizer lo hace de manera automática (agrega eso si el token #).

In [10]:
print(tokenizer("#UnHashtag",truncation=True))
print(tokenizer("Un Hashtag",truncation=True))

{'input_ids': [4, 3, 1044, 20247, 5001, 3483, 5], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}
{'input_ids': [4, 1044, 1354, 5001, 3483, 5], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}


In [11]:
tokenized_dataset = dataset_emoji.map(preprocess_function, batched=True)

Parameter 'function'=<function preprocess_function at 0x7f8bf25f7af0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
100%|██████████| 10/10 [00:00<00:00, 25.97ba/s]


In [12]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Ahora importamos el modelo

In [13]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=19)

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuc

y el trainer de transformers

In [14]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

Como primer test, ejecutamos una sola epoca.

**IMPORTANTE:** en esta versión sólo entrenamos una vuelta sobre el conjunto trial.

In [15]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/camilo/miniconda3/envs/datamining_env/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 10000
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 625
 80%|████████  | 500/625 [02:00<00:31,  3.95it/s]Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/con

{'loss': 2.5416, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.8}


Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
100%|██████████| 625/625 [02:33<00:00,  4.18it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 625/625 [02:33<00:00,  4.07it/s]

{'train_runtime': 153.7477, 'train_samples_per_second': 65.042, 'train_steps_per_second': 4.065, 'train_loss': 2.520683837890625, 'epoch': 1.0}


TrainOutput(global_step=625, training_loss=2.520683837890625, metrics={'train_runtime': 153.7477, 'train_samples_per_second': 65.042, 'train_steps_per_second': 4.065, 'train_loss': 2.520683837890625, 'epoch': 1.0})

Nos hemos demorado 2 minutos y medio.

### Acelerando el entrenamiento con GPU

En el procedimiento anterior se uso solamente la CPU. Podemos acelerar nuestro entramiento usando una targeta gráfica de Nvidia. Seguiremos [este tutorial](https://huggingface.co/docs/transformers/perf_train_gpu_one). Con respecto a lo anterior aplicamos un reseteo de la kernel, solo por consistencia, por ende cargamos lo realizado anteriormente.

In [1]:
from config import *
from datasets import ClassLabel
from datasets import load_dataset
from transformers import AutoTokenizer

data_files = {"train": file_names['df_es_trial'], "test": file_names['df_es_test']}  # a modo de testeo no entrenaremos en el train set entero
dataset_emoji = load_dataset("pandas", data_files=data_files)
dataset_emoji = dataset_emoji.remove_columns("id")
dataset_emoji = dataset_emoji.cast_column("label",ClassLabel(num_classes=19))

model_id = "dccuchile/bert-base-spanish-wwm-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_id)

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)
tokenized_dataset = dataset_emoji.map(preprocess_function, batched=True)

from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Using custom data configuration default-cb077f95137f42ce
Reusing dataset pandas (/home/camilo/.cache/huggingface/datasets/pandas/default-cb077f95137f42ce/0.0.0/6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade)
100%|██████████| 2/2 [00:00<00:00, 540.09it/s]
Loading cached processed dataset at /home/camilo/.cache/huggingface/datasets/pandas/default-cb077f95137f42ce/0.0.0/6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade/cache-d9b1254ac31c380b.arrow
Loading cached processed dataset at /home/camilo/.cache/huggingface/datasets/pandas/default-cb077f95137f42ce/0.0.0/6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade/cache-e3006495028cdd71.arrow
Parameter 'function'=<function preprocess_function at 0x7f4e92a248b0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and cachi

Las siguientes funciones nos muestran el uso de GPU.

In [2]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [3]:
print_gpu_utilization()

GPU memory occupied: 49 MB.


Cuando carguemos el modelo esta vez lo haremos hacia la GPU, lo cual usará espacio en ella.

In [4]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=19).to("cuda")
print_gpu_utilization()

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuc

GPU memory occupied: 1271 MB.


In [8]:
import bitsandbytes as bnb
from torch import nn
from transformers.trainer_pt_utils import get_parameter_names
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    gradient_accumulation_steps=4, 
    gradient_checkpointing=True
)

decay_parameters = get_parameter_names(model, [nn.LayerNorm])
decay_parameters = [name for name in decay_parameters if "bias" not in name]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if n in decay_parameters],
        "weight_decay": training_args.weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if n not in decay_parameters],
        "weight_decay": 0.0,
    },
]

optimizer_kwargs = {
    "betas": (training_args.adam_beta1, training_args.adam_beta2),
    "eps": training_args.adam_epsilon,
}
optimizer_kwargs["lr"] = training_args.learning_rate
adam_bnb_optim = bnb.optim.Adam8bit(
    optimizer_grouped_parameters,
    betas=(training_args.adam_beta1, training_args.adam_beta2),
    eps=training_args.adam_epsilon,
    lr=training_args.learning_rate,
)

In [12]:
from accelerate import Accelerator
from torch.utils.data.dataloader import DataLoader

dataloader = DataLoader(tokenized_dataset['train'], batch_size=training_args.per_device_train_batch_size)

if training_args.gradient_checkpointing:
    model.gradient_checkpointing_enable()

accelerator = Accelerator(fp16=training_args.fp16)
model, optimizer, dataloader = accelerator.prepare(model, adam_bnb_optim, dataloader)

model.train()
for step, batch in enumerate(dataloader, start=1):
    loss = model(**batch).loss
    loss = loss / training_args.gradient_accumulation_steps
    accelerator.backward(loss)
    if step % training_args.gradient_accumulation_steps == 0:
        optimizer.step()
        optimizer.zero_grad()

TypeError: forward() got an unexpected keyword argument 'text'

In [ ]:
print_gpu_utilization()

# ---

In [5]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    gradient_accumulation_steps=4, 
    gradient_checkpointing=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

Using amp half precision backend


In [6]:
result = trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/camilo/miniconda3/envs/datamining_env/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 10000
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 4
  Total optimization steps = 156
  0%|          | 0/156 [00:00<?, ?it/s]

RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling `cublasGemmEx( handle, opa, opb, m, n, k, &falpha, a, CUDA_R_16F, lda, b, CUDA_R_16F, ldb, &fbeta, c, CUDA_R_16F, ldc, CUDA_R_32F, CUBLAS_GEMM_DFALT_TENSOR_OP)`

In [ ]:
print_summary(result)